# Semantic Search with Semantic Text

<a target="_blank" href="https://colab.research.google.com/github/Mikep86/elasticsearch-labs/blob/semantic-text-notebook/notebooks/search/09-semantic-text.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Learn how to use the [semantic_text](https://www.elastic.co/guide/en/elasticsearch/reference/master/semantic-text.html) field type to quickly get started with semantic search.

## Requirements

For this example, you will need:

- An Elastic deployment:
  - We'll be using [Elastic Cloud](https://www.elastic.co/guide/en/cloud/current/ec-getting-started.html) for this example (available with a [free trial](https://cloud.elastic.co/registration?utm_source=github&utm_content=elasticsearch-labs-notebook))

- Elasticsearch 8.15 or above, or [Elasticsearch serverless](https://www.elastic.co/elasticsearch/serverless)

## Create Elastic Cloud deployment

If you don't have an Elastic Cloud deployment, sign up [here](https://cloud.elastic.co/registration?utm_source=github&utm_content=elasticsearch-labs-notebook) for a free trial.

## Install packages and connect with Elasticsearch Client

To get started, we'll need to connect to our Elastic deployment using the Python client (version 8.15.0 or above).
Because we're using an Elastic Cloud deployment, we'll use the **Cloud ID** to identify our deployment.

First we need to `pip` install the following packages:

- `elasticsearch`

In [ ]:
!pip install elasticsearch

Next, we need to import the modules we need. 

🔐 NOTE: getpass enables us to securely prompt the user for credentials without echoing them to the terminal, or storing it in memory.

In [ ]:
from elasticsearch import Elasticsearch, exceptions
from urllib.request import urlopen
from getpass import getpass
import json
import time

Now we can instantiate the Python Elasticsearch client.

First we prompt the user for their password and Cloud ID.
Then we create a `client` object that instantiates an instance of the `Elasticsearch` class.

In [ ]:
# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#finding-your-cloud-id
ELASTIC_CLOUD_ID = getpass("Elastic Cloud ID: ")

# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#creating-an-api-key
ELASTIC_API_KEY = getpass("Elastic Api Key: ")

# Create the client instance
client = Elasticsearch(
    # For local development
    # hosts=["http://localhost:9200"]
    cloud_id=ELASTIC_CLOUD_ID,
    api_key=ELASTIC_API_KEY,
)

### Enable Telemetry

Knowing that you are using this notebook helps us decide where to invest our efforts to improve our products. We would like to ask you that you run the following code to let us gather anonymous usage statistics. See [telemetry.py](https://github.com/elastic/elasticsearch-labs/blob/main/telemetry/telemetry.py) for details. Thank you!

In [ ]:
!curl -O -s https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/telemetry/telemetry.py
from telemetry import enable_telemetry

client = enable_telemetry(client, "09-semantic-text")

### Test the Client
Before you continue, confirm that the client has connected with this test.

In [ ]:
print(client.info())

Refer to [the documentation](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html#connect-self-managed-new) to learn how to connect to a self-managed deployment.

Read [this page](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html#connect-self-managed-new) to learn how to connect using API keys.

## Create the Inference Endpoint

Let's create the inference endpoint by using the [Create inference API](https://www.elastic.co/guide/en/elasticsearch/reference/current/put-inference-api.html).

For this example we'll use the [ELSER service](https://www.elastic.co/guide/en/machine-learning/current/ml-nlp-elser.html), but the inference API also supports [many other inference services](https://www.elastic.co/guide/en/elasticsearch/reference/current/put-inference-api.html#put-inference-api-desc).

In [ ]:
try:
    client.inference.delete_model(inference_id="my-elser-endpoint")
except exceptions.NotFoundError:
    # Inference endpoint does not exist
    pass

try:
    client.options(request_timeout=60, max_retries=3, retry_on_timeout=True).inference.put_model(
        task_type="sparse_embedding",
        inference_id="my-elser-endpoint",
        body={
            "service": "elser",
            "service_settings": {"num_allocations": 1, "num_threads": 1},
        },
    )
    print("Inference endpoint created successfully")
except exceptions.BadRequestError as e:
    if e.error == "resource_already_exists_exception":
        print("Inference endpoint created successfully")
    else:
        raise e


Once the endpoint is created, we must wait until the backing ELSER service is deployed.
This can take a few minutes to complete.

In [ ]:
inference_endpoint_info = client.inference.get_model(
    inference_id="my-elser-endpoint",
)
model_id = inference_endpoint_info["endpoints"][0]["service_settings"]["model_id"]

while True:
    status = client.ml.get_trained_models_stats(
        model_id=model_id,
    )

    deployment_stats = status["trained_model_stats"][0].get("deployment_stats")
    if deployment_stats is None:
        print("ELSER Model is currently being deployed.")
        continue
    
    nodes = deployment_stats.get("nodes")
    if nodes is not None and len(nodes) > 0:
        print("ELSER Model has been successfully deployed.")
        break
    else:
        print("ELSER Model is currently being deployed.")
    time.sleep(5)

## Create the Index

Now we need to create an index with a `semantic_text` field. Let's create one that enables us to perform semantic search on movie plots.

In [ ]:
client.indices.delete(index="semantic-text-movies", ignore_unavailable=True)
client.indices.create(
    index="semantic-text-movies",
    mappings={
        "properties": {
            "title": {"type": "text"},
            "genre": {"type": "text"},
            "plot": {"type": "text", "copy_to": "plot_semantic"},
            "plot_semantic": {
                "type": "semantic_text",
                "inference_id": "my-elser-endpoint",
            },
        }
    },
)

Notice how we configured the mappings. We defined `plot_semantic` as a `semantic_text` field.
The `inference_id` parameter defines the inference endpoint that is used to generate the embeddings for the field.
Then we configured the `plot` field to [copy its value](https://www.elastic.co/guide/en/elasticsearch/reference/current/copy-to.html) to the `plot_semantic` field.

## Populate the Index

Let's populate the index with our example dataset of 12 movies.

In [ ]:
url = "https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/notebooks/search/movies.json"
response = urlopen(url)
movies = json.loads(response.read())

operations = []
for movie in movies:
    operations.append({"index": {"_index": "semantic-text-movies"}})
    operations.append(movie)
client.bulk(index="semantic-text-movies", operations=operations, refresh=True)

## Semantic Search

Now that our index is populated, we can query it using semantic search.

### Aside: Pretty printing Elasticsearch search results

Your `search` API calls will return hard-to-read nested JSON.
We'll create a little function called `pretty_search_response` to return nice, human-readable outputs from our examples.

In [ ]:
def pretty_search_response(response):
    if len(response["hits"]["hits"]) == 0:
        print("Your search returned no results.")
    else:
        for hit in response["hits"]["hits"]:
            id = hit["_id"]
            score = hit["_score"]
            title = hit["_source"]["title"]
            runtime = hit["_source"]["runtime"]
            plot = hit["_source"]["plot"]
            keyScene = hit["_source"]["keyScene"]
            genre = hit["_source"]["genre"]
            released = hit["_source"]["released"]

            pretty_output = f"\nID: {id}\nScore: {score}\nTitle: {title}\nRuntime: {runtime}\nPlot: {plot}\nKey Scene: {keyScene}\nGenre: {genre}\nReleased: {released}"

            print(pretty_output)

### Semantic Search with the `semantic` Query

We can use the [`semantic` query](https://www.elastic.co/guide/en/elasticsearch/reference/master/query-dsl-semantic-query.html) to quickly & easily query the `semantic_text` field in our index.
Under the hood, an embedding is automatically generated for our query text using the `semantic_text` field's inference endpoint.

In [ ]:
response = client.search(
    index="semantic-text-movies",
    query={"semantic": {"field": "plot_semantic", "query": "organized crime movies"}},
)

pretty_search_response(response)

These results demonstrate the power of semantic search.
Our top results are all movies involving organized crime, even if the exact term "organized crime" doesn't appear in the plot description.
This works because the ELSER model understands the semantic similarity between terms like "organized crime" and "mob".

However, these results also show the weaknesses of semantic search.
Because semantic search is based on vector similarity, there is a long tail of results that are weakly related to our query vector.
That's why movies like _The Matrix_ are returned towards the tail end of our search results.

### Hybrid Search with the `semantic` Query

We can address some of the issues with pure semantic search by combining it with lexical search techniques.
Here, we use a [boolean query](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-bool-query.html) to require that all matches contain at least term from the query text, in either the `plot` or `genre` fields.

In [ ]:
response = client.search(
    index="semantic-text-movies",
    query={
        "bool": {
            "must": {
                "multi_match": {
                    "fields": ["plot", "genre"],
                    "query": "organized crime movies",
                }
            },
            "should": {
                "semantic": {
                    "field": "plot_semantic",
                    "query": "organized crime movies",
                }
            },
        }
    },
)

pretty_search_response(response)

These results demonstrate that the application of lexical search techniques can help focus the results, while retaining many of the advantages of semantic search.
In this example, the top search results are all still movies involving organized crime, but the `multi_match` query keeps the long tail shorter and focused on movies in the crime genre.

## Conclusion

The [semantic_text](https://www.elastic.co/guide/en/elasticsearch/reference/master/semantic-text.html) field type is a powerful tool that can help you quickly and easily integrate semantic search.
It can greatly improve the relevancy of your search results, particularly when combined with lexical search techniques.